In [2]:
import os
os.chdir('../../')

In [3]:
import numpy as np
import pandas as pd

from scr.util import *
from scr.feature_engineering.engineering import *

In [8]:
df_train = pd.read_csv('data/feature_engineered/null_cat/train_null_cat.csv')
df_test = pd.read_csv('data/feature_engineered/null_cat/test_null_cat.csv')

In [9]:
display(df_train.iloc[:5, :21])
display(df_test.iloc[:5, :21])

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child,AgeGroup
0,0,50,Self Enquiry,2,6.803505,Large Business,male,1,4,Basic,...,5,1,4,Executive,253905.0,1,Single,No Car,0_child,50s
1,1,56,Company Invited,1,6.734592,Salaried,male,1,4,Standard,...,2,1,4,Senior Manager,404475.0,0,Divorced,Has Car,0_child,50s
2,2,32,Self Enquiry,1,6.398595,Large Business,female,1,3,Basic,...,4,0,4,Executive,278145.0,1,Married,No Car,0_child,30s
3,3,37,Self Enquiry,2,6.985642,Small Business,female,1,3,Standard,...,1,0,5,Senior Manager,326805.0,0,Divorced,Has Car,0_child,30s
4,4,48,Company Invited,3,6.928538,Small Business,female,1,3,Basic,...,4,0,4,Executive,258435.0,1,Single,Has Car,0_child,40s


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,Marry,Car,Child,AgeGroup,NumberOfTrips_log
0,3489,48,Self Enquiry,2,6.660575,Small Business,male,1,4,Super Deluxe,...,7,0,3,AVP,496950.0,Married,Has Car,0_child,40s,2.079442
1,3490,30,Self Enquiry,2,6.580639,Small Business,female,1,4,Standard,...,4,1,3,Senior Manager,300000.0,Married,No Car,0_child,30s,1.609438
2,3491,25,Self Enquiry,1,6.293419,Salaried,female,1,4,Basic,...,1,0,3,Executive,260000.0,Divorced,No Car,0_child,20s,0.693147
3,3492,21,Company Invited,2,6.042633,Salaried,male,1,4,Basic,...,1,0,3,Senior Manager,259875.0,Divorced,Has Car,0_child,20s,0.693147
4,3493,41,Company Invited,1,6.042633,Salaried,male,1,4,Basic,...,1,0,4,Executive,268830.0,Single,Has Car,0_child,40s,0.693147


# 特徴量作成

## TypeOfContactNuLL
TypeOfContact が欠損値の場合を、明らかに学習させるための特徴量

In [10]:
df_train['TypeofContactNULL'] = df_train['TypeofContact'].apply(make_TypeOfContactNULL)
df_test['TypeofContactNULL'] = df_test['TypeofContact'].apply(make_TypeOfContactNULL)

## Motivation
顧客の旅行に対する意欲を表す特徴量

In [11]:
df_train['Motivation'] = df_train['NumberOfPersonVisiting'] * df_train['NumberOfFollowups'] + (df_train['Passport'] * 10)
df_test['Motivation'] = df_test['NumberOfPersonVisiting'] * df_test['NumberOfFollowups'] + (df_test['Passport'] * 10)

## EconomicPower
顧客の経済力を表す特徴量

In [12]:
df_train['EconomicPower'] = df_train['MonthlyIncome'] * (4 - df_train['CityTier']) * df_train['Gender'].apply(make_motivation_gender)
df_test['EconomicPower'] = df_test['MonthlyIncome'] * (4 - df_test['CityTier']) * df_test['Gender'].apply(make_motivation_gender)

## Child01
顧客に子どもがいるかどうかを表す特徴量

In [13]:
df_train['Child01'] = df_train['Child'].apply(make_child01)
df_test['Child01'] = df_test['Child'].apply(make_child01)

## TripEasier
旅行の行きやすさを表す特徴量

In [14]:
df_train['TripEasier'] = (5 - df_train['NumberOfPersonVisiting']) * df_train['ProductPitched'].apply(make_TripEasier_ProductPitched) * df_train['Marry'].apply(make_TripEasier_Marry) * df_train['Child'].apply(make_TripEasier_Child)
df_test['TripEasier'] = (5 - df_test['NumberOfPersonVisiting']) * df_test['ProductPitched'].apply(make_TripEasier_ProductPitched) * df_test['Marry'].apply(make_TripEasier_Marry) * df_test['Child'].apply(make_TripEasier_Child)

## SalesPerformance
営業担当者の単位時間当たりのパフォーマンス

In [15]:
df_train['SalesPerformance'] = df_train['NumberOfFollowups'] * df_train['PitchSatisfactionScore'] / df_train['DurationOfPitch']
df_test['SalesPerformance'] = df_train['NumberOfFollowups'] * df_test['PitchSatisfactionScore'] / df_test['DurationOfPitch']

## LivingCost
生活コストを表す数値

In [16]:
df_train['LivingCost'] = df_train['MonthlyIncome'] / (df_train['CityTier'].apply(make_LivingCost_CityTier) * (1 + df_train['Child'].apply(make_LivingCost_Child) + df_train['Marry'].apply(make_LivingCost_Marry) + df_train['Car'].apply(make_LivingCost_Car)))
df_test['LivingCost'] = df_test['MonthlyIncome'] / (df_test['CityTier'].apply(make_LivingCost_CityTier) * (1 + df_test['Child'].apply(make_LivingCost_Child) + df_test['Marry'].apply(make_LivingCost_Marry) + df_test['Car'].apply(make_LivingCost_Car)))

## EconomicStability
経済安定性指標

In [17]:
df_train['EconomicStability'] = df_train['MonthlyIncome'] * df_train['Occupation'].apply(make_EconomicStability_Occupation) * df_train['Designation'].apply(make_EconomicStability_Disignation)
df_test['EconomicStability'] = df_test['MonthlyIncome'] * df_test['Occupation'].apply(make_EconomicStability_Occupation) * df_test['Designation'].apply(make_EconomicStability_Disignation)

## TripFreaqency
旅行頻度

In [18]:
df_train['TripFreaqency'] = df_train['NumberOfTrips'].apply(make_TripFreaqency)
df_test['TripFreaqency'] = df_test['NumberOfTrips'].apply(make_TripFreaqency)

## NumberOfTrips_log

In [19]:
df_train['NumberOfTrips_log'] = np.log1p(df_train['NumberOfTrips'])
df_test['NumberOfTrips_log'] = np.log1p(df_test['NumberOfTrips'])

## TravelCost
総旅行費の推定

In [20]:
df_train['TravelCost'] = df_train['PreferredPropertyStar'].apply(make_TravelCost_PreferredPropertyStar) * (1 + df_train['NumberOfTrips_log'])
df_test['TravelCost'] = df_test['PreferredPropertyStar'].apply(make_TravelCost_PreferredPropertyStar) * (1 + df_test['NumberOfTrips_log'])

## EconomicSegment
経済力のセグメント

In [21]:
df_train['EconomicSegment'] = df_train['Occupation'] + df_train['Designation'] + df_train['CityTier'].astype(str)
df_test['EconomicSegment'] = df_test['Occupation'] + df_test['Designation'] + df_test['CityTier'].astype(str)

df_train = mapping_EconomicSegment(df_train)
df_test = mapping_EconomicSegment(df_test)

## PacakgeMatch
営業担当者がセールスした商品と、顧客の希望するホテルのランクがマッチしているかを表す数値

In [22]:
product_to_star = {
    'Basic': 3,
    'Standard': 3,
    'Deluxe': 4,
    'Super Deluxe': 5,
    'King': 5
}

# 製品と希望ホテルの星評価の適合度を計算
def calculate_fit(product, preferred_star):
    ideal_star = product_to_star[product]
    max_star_difference = 5  # 星評価の最大差（1星から5星）
    return max(0, 1 - abs(ideal_star - preferred_star) / max_star_difference)

# 新しい特徴量をデータフレームに追加
df_train['PackageMatch'] = df_train.apply(lambda row: calculate_fit(row['ProductPitched'], row['PreferredPropertyStar']), axis=1)
df_test['PackageMatch'] = df_test.apply(lambda row: calculate_fit(row['ProductPitched'], row['PreferredPropertyStar']), axis=1)

In [23]:
df_train[['PackageMatch', 'ProductPitched', 'PreferredPropertyStar']]

,PackageMatch,ProductPitched,PreferredPropertyStar
0,1.0,Basic,3
1,1.0,Standard,3
2,1.0,Basic,3
3,0.8,Standard,4
4,0.8,Basic,4
...,...,...,...
3484,1.0,Basic,3
3485,0.6,Basic,5
3486,1.0,Standard,3
3487,0.6,King,3


## Family
家族か、そうでないか、2人以上ならば家族とする。

In [32]:
df_train['IsFamily'] = df_train.apply(lambda x: int(x['Marry'] == 'Married' or x['Child'] != '0_child'), axis=1)
df_test['IsFamily'] = df_test.apply(lambda x: int(x['Marry'] == 'Married' or x['Child'] != '0_child'), axis=1)

## IncomeGrouped
顧客セグメント毎の1人当たりの収入（年齢、性別、都市層）

In [40]:
tmp_train = df_train.groupby(by=['AgeGroup', 'Gender', 'CityTier']).agg(
    count = ('id', 'count'),
    total = ('MonthlyIncome', 'sum')
)
tmp_train['IncomeGrouped'] = tmp_train['total'] / tmp_train['count']

tmp_test = df_test.groupby(by=['AgeGroup', 'Gender', 'CityTier']).agg(
    count = ('id', 'count'),
    total = ('MonthlyIncome', 'sum')
)
tmp_test['IncomeGrouped'] = tmp_test['total'] / tmp_test['count']

In [42]:
df_train['MonthlyIncome']

0       253905.0
1       404475.0
2       278145.0
3       326805.0
4       258435.0
          ...   
3484    258900.0
3485    260415.0
3486    317340.0
3487    527910.0
3488    278190.0
Name: MonthlyIncome, Length: 3489, dtype: float64

In [41]:
tmp_train

count         total  IncomeGrouped
AgeGroup Gender CityTier                                    
10s      female 1             2  5.670100e+05  283505.000000
                2             2  6.768450e+05  338422.500000
                3             1  2.630400e+05  263040.000000
         male   1             6  1.814975e+06  302495.833333
                2             7  2.072660e+06  296094.285714
20s      female 1            65  1.920819e+07  295510.655086
                2            96  2.963766e+07  308725.600416
                3            43  1.315892e+07  306021.452611
         male   1           111  3.438202e+07  309747.882883
                2           211  6.752369e+07  320017.488152
                3            35  1.155596e+07  330170.142857
30s      female 1           235  7.113894e+07  302718.910131
                2           219  6.994669e+07  319391.256785
                3            63  1.956243e+07  310514.724401
         male   1           324  1.036479e+08  319900.990206
                2           374  1.269264e+08  339375.328869
                3           104  3.585857e+07  344793.971200
40s      female 1           158  5.633740e+07  356565.812047
                2           102  3.649232e+07  357767.794118
                3            30  1.106030e+07  368676.666667
         male   1           267  9.834054e+07  368316.626053
                2           297  1.111735e+08  374321.515152
                3            67  2.543782e+07  379669.029851
50s      female 1           102  4.159503e+07  407794.411765
                2           111  4.331333e+07  390210.159544
                3            27  1.038537e+07  384643.333333
         male   1           189  7.878894e+07  416872.712898
                2           175  7.277798e+07  415874.187445
                3            41  1.647804e+07  401903.414634
60s      female 1             5  2.242705e+06  448541.000000
                3             3  1.143600e+06  381200.000000
         male   1            11  5.103840e+06  463985.454545
                2             2  7.840900e+05  392045.000000
                3             4  1.623835e+06  405958.750000

# 特徴量のデータ型

In [130]:
feature = [
    'Age',
    'TypeofContact',
    'CityTier',
    'DurationOfPitch',
    'Occupation',
    'Gender',
    'NumberOfPersonVisiting',
    'NumberOfFollowups',
    'ProductPitched',
    'PreferredPropertyStar',
    'NumberOfTrips',
    'Passport',
    'PitchSatisfactionScore',
    'Designation',
    'MonthlyIncome',
    'Marry',
    'Car',
    'Child',
    # 以下、作成特徴量
    'AgeGroup',
    'TypeofContactNULL',
    'Motivation',
    'EconomicPower',
    'Child01',
    'TripEasier',
    'SalesPerformance',
    'LivingCost',
    'EconomicStability',
    'TripFreaqency',
    'TravelCost',
    'NumberOfTrips_log',
    'EconomicSegment',
    'PackageMatch'
]

In [131]:
float_columns = ['DurationOfPitch', 'MonthlyIncome', 'EconomicPower', 'SalesPerformance',
                'LivingCost', 'EconomicStability', 'TravelCost', 'NumberOfTrips_log',
                'PackageMatch']
int_columns = ['Age', 'CityTier', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'PreferredPropertyStar',
                'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'TypeofContactNULL',
                'Motivation', 'Child01', 'TripEasier', 'TripFreaqency', 'EconomicSegment'
                ]

In [132]:
for col in float_columns:
    df_train[col] = df_train[col].astype(float)
    df_test[col] = df_test[col].astype(float)

for col in int_columns:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

# 確認

In [24]:
display(df_train.iloc[:5, 20:])
display(df_test.iloc[:5, 19:])

,AgeGroup,NumberOfTrips_log,TypeofContactNULL,Motivation,EconomicPower,Child01,TripEasier,SalesPerformance,LivingCost,EconomicStability,TripFreaqency,TravelCost,EconomicSegment,PackageMatch
0,50s,1.791759,0,14,660153.0,0,48.0,2.351729,195311.538462,335154.60,1,2.791759,0,1.0
1,50s,1.098612,0,14,1577452.5,0,8.0,2.375794,256809.523810,578399.25,0,2.098612,1,1.0
2,30s,1.609438,0,3,834435.0,0,24.0,1.875412,168572.727273,367151.40,0,2.609438,2,1.0
3,30s,0.693147,0,3,653610.0,0,8.0,2.147262,239417.582418,382361.85,0,2.539721,3,0.8
4,40s,1.609438,0,3,258435.0,0,48.0,1.731967,246128.571429,255850.65,0,3.914157,4,0.8


,AgeGroup,NumberOfTrips_log,TypeofContactNULL,Motivation,EconomicPower,Child01,TripEasier,SalesPerformance,LivingCost,EconomicStability,TripFreaqency,TravelCost,EconomicSegment,PackageMatch
0,40s,2.079442,0,4,1292070.0,0,0.008,1.801646,332408.026756,626157.00,1,3.079442,10,0.6
1,30s,1.609438,0,14,600000.0,0,8.000,1.823531,209790.209790,351000.00,0,2.609438,3,1.0
2,20s,0.693147,0,4,780000.0,0,24.000,1.430065,173333.333333,314600.00,0,1.693147,13,1.0
3,20s,0.693147,0,4,675675.0,0,24.000,1.489417,190384.615385,371621.25,0,2.539721,20,0.8
4,40s,0.693147,0,4,1048437.0,0,48.000,1.985889,170685.714286,325284.30,0,1.693147,13,1.0


# CSV出力

In [134]:
if 'test' in df_train.columns:
    df_train = df_train.drop(columns='test', axis=1)

In [135]:
df_train.to_csv('data/feature_engineered/null_del_xxx/train_null_del.csv', index=False)
df_test.to_csv('data/feature_engineered/null_del_xxx/test_null_median.csv', index=False)